# Quick Introduction

In this project, we aim to predict the price of bitcoin using sentiment analysis and finalcial features to support the sentiment.

We will compute the error (Mean absolute error of two models we developed to predict the prices)

The two models are:
1. Random Forrest Regression
2. Linear Regression

The sentiment was detected using Vader, used to detect polarity

In [ ]:
#import libraries to import the data
import pandas as pd
import numpy as np
import re
!pip install clean-text

In [104]:
# Since Elon only started tweeting about crypto from 2019, lets only include tweets from 2019,2020,2021,2022

elon_tweets_2019 = pd.read_csv('2019.csv')
elon_tweets_2020 = pd.read_csv('2020.csv')
elon_tweets_2021 = pd.read_csv('2021.csv')
elon_tweets_2022 = pd.read_csv('2022.csv')

# we would also be forcasting in bitcoin tweets from 2021 on general, to make our results more generalizable. we chose 2021
# because the volume of bitcoin tweets in 2021 has skyrocketed.

bitcoin_tweets_2021 = pd.read_csv('Bitcoin_tweets.csv')

/Users/arjun/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [105]:
# combining all tweets from elon musk 2019-2022
elon_tweets = pd.concat([elon_tweets_2019, elon_tweets_2020, elon_tweets_2021,elon_tweets_2022], axis = 0)
elon_tweets = elon_tweets.reset_index(drop=True)

/Users/arjun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [4]:

elon_tweets

,Unnamed: 0,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,...,trans_src,translate,tweet,urls,user_id,user_id_str,user_rt,user_rt_id,username,video
0,0.0,[],1210918835861241856,1.57758e+12,2019-12-28 23:47:59,6.0,NaN,[],23.0,1211071324518531072,...,NaN,NaN,@Joe__Wakefield @austinbarnard45 @tjq1190 @tyg...,[],44196397,44196397.0,NaN,NaN,elonmusk,0
1,1.0,[],1210918835861241856,1.57758e+12,2019-12-28 23:38:57,6.0,NaN,[],23.0,1211069054779301894,...,NaN,NaN,@austinbarnard45 @tjq1190 @tyger_cyber @fawful...,[],44196397,44196397.0,NaN,NaN,elonmusk,0
2,2.0,[],1210918835861241856,1.57758e+12,2019-12-28 23:22:36,6.0,NaN,[],23.0,1211064937004589056,...,NaN,NaN,@IrinaAntony @tjq1190 @tyger_cyber @fawfulfan ...,[],44196397,44196397.0,NaN,NaN,elonmusk,0
3,3.0,[],1210918835861241856,1.57757e+12,2019-12-28 22:42:53,6.0,NaN,[],22.0,1211054942192119808,...,NaN,NaN,@tjq1190 @tyger_cyber @fawfulfan @_Mikemo He d...,[],44196397,44196397.0,NaN,NaN,elonmusk,0
4,4.0,[],1210774027054145536,1.57757e+12,2019-12-28 22:30:09,6.0,NaN,[],22.0,1211051740562366464,...,NaN,NaN,@geofficient Pretty much,[],44196397,44196397.0,NaN,NaN,elonmusk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24167,NaN,[],1477775713172402176,2022-01-03 02:56:00 Arabian Standard Time,2022-01-03,NaN,NaN,[],NaN,1477775713172402176,...,NaN,NaN,https://t.co/LA9hPzVlGx,[],44196397,NaN,NaN,NaN,elonmusk,1
24168,NaN,[],1477706142461706248,2022-01-02 22:19:33 Arabian Standard Time,2022-01-02,NaN,NaN,[],NaN,1477706142461706248,...,NaN,NaN,Let’s make the roaring 20’s happen!,[],44196397,NaN,NaN,NaN,elonmusk,0
24169,NaN,[],1477700424933253123,2022-01-02 21:56:50 Arabian Standard Time,2022-01-02,NaN,NaN,[],NaN,1477700424933253123,...,NaN,NaN,Great work by Tesla team worldwide!,[],44196397,NaN,NaN,NaN,elonmusk,0
24170,NaN,[],1476958174439567360,2022-01-01 05:58:52 Arabian Standard Time,2022-01-01,NaN,NaN,[],NaN,1477096955658579974,...,NaN,NaN,@BLKMDL3 @Tesla 🔥,[],44196397,NaN,NaN,NaN,elonmusk,0


In [5]:
elon_tweets.describe()

,Unnamed: 0,conversation_id,day,geo,hour,id,likes_count,near,nlikes,nreplies,...,source,timezone,trans_dest,trans_src,translate,user_id,user_id_str,user_rt,user_rt_id,video
count,20029.000000,2.417200e+04,20029.000000,0.0,20029.000000,2.417200e+04,4143.000000,0.0,2.002900e+04,20029.000000,...,0.0,24172.000000,0.0,0.0,0.0,24172.0,20029.0,0.0,0.0,24172.000000
mean,5151.465475,1.059613e+18,4.135054,NaN,12.445504,1.061956e+18,49953.278059,NaN,1.596744e+04,452.085176,...,NaN,68.558663,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,0.063627
std,3128.308904,3.026564e+17,1.969430,NaN,7.767980,3.008595e+17,103849.982613,NaN,5.119675e+04,1547.741292,...,NaN,150.745199,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.244093
min,0.000000,1.659576e+09,1.000000,NaN,0.000000,1.543473e+10,850.000000,NaN,2.000000e+00,0.000000,...,NaN,0.000000,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,0.000000
25%,2503.000000,9.178526e+17,2.000000,NaN,5.000000,9.215870e+17,5109.000000,NaN,7.170000e+02,38.000000,...,NaN,0.000000,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,0.000000
50%,5007.000000,1.108829e+18,4.000000,NaN,15.000000,1.110576e+18,10398.000000,NaN,1.874000e+03,92.000000,...,NaN,0.000000,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,0.000000
75%,7510.000000,1.266508e+18,6.000000,NaN,19.000000,1.269111e+18,33697.500000,NaN,9.082000e+03,306.000000,...,NaN,0.000000,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,0.000000
max,11716.000000,1.499977e+18,7.000000,NaN,23.000000,1.500030e+18,963257.000000,NaN,1.596525e+06,45699.000000,...,NaN,400.000000,NaN,NaN,NaN,44196397.0,44196397.0,NaN,NaN,1.000000


In [106]:
# we only need the tweet and the data of the tweet for analysis

elon_tweets = elon_tweets[['date','tweet']]
elon_tweets

,date,tweet
0,2019-12-28 23:47:59,@Joe__Wakefield @austinbarnard45 @tjq1190 @tyg...
1,2019-12-28 23:38:57,@austinbarnard45 @tjq1190 @tyger_cyber @fawful...
2,2019-12-28 23:22:36,@IrinaAntony @tjq1190 @tyger_cyber @fawfulfan ...
3,2019-12-28 22:42:53,@tjq1190 @tyger_cyber @fawfulfan @_Mikemo He d...
4,2019-12-28 22:30:09,@geofficient Pretty much
...,...,...
24167,2022-01-03,https://t.co/LA9hPzVlGx
24168,2022-01-02,Let’s make the roaring 20’s happen!
24169,2022-01-02,Great work by Tesla team worldwide!
24170,2022-01-01,@BLKMDL3 @Tesla 🔥


In [107]:
pd.options.mode.chained_assignment = None
# let's remove the timestamp from the date field and only include the date
elon_tweets.date = elon_tweets.date.str[:10] #tweets.date.str[:10]


In [108]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z0-9]", " ")
    tweets = np.core.defchararray.replace(tweets, "#", " ")
    tweets = np.core.defchararray.replace(tweets, "$", " ")
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
   
    
    return tweets

In [109]:
elon_tweets['tweet'] = clean_tweets(elon_tweets['tweet'])

In [110]:
elon_tweets.reset_index(inplace=True, drop=True)

In [111]:
elon_tweets['tweet'].head(50)

0           This is a pretty awful lie. I left South...
1          This person blocked me, so can’t read the...
2          We started Zip2 with ~ 2k from me plus my...
3         He didn’t own an emerald mine &amp; I work...
4                                           Pretty much
5                                               Federal
6       Probably 2021. Depends on regulatory approvals.
7      Hopefully working in Caribbean by end of 2020...
8       Maximizing conversion of mass to aimed photo...
9                              Yes, but it’s too secret
10                  Hopefully fully operational in 2020
11      Boring Co is completing its first commercial...
12                         No, he’s still much faster 🐌
13    These would be road tunnels for zero emissions...
14                                         Done by dawn
15                    That would be a fringe benefit 🤣🤣
16    Build super safe, Earthquake-proof tunnels und...
17               Flight is hopefully 2 to 3 mont

NameError: name 'elon_bit_doge' is not defined

In [112]:
elon_tweets['tweet'] = elon_tweets['tweet'].str.lower()
elon_tweets['tweet'] = elon_tweets['tweet'].str.lstrip()






In [113]:
# after manually browsing the internet for crytocurrency hashtags and keywords, we found a comprehensive list of words listed below
crpyto_keywords = ["#Bitcoin","#BTC","#Cryptocurrency","#Crypto",'Bitcoin','BTC',"#Ethereum","Ethereum", "DOGE","dogecoin",'doge','shibainu','shiba','floki','snl',
                   'SNL', 'HOLD','hold','HODL','hodl','BabyDoge','babydoge']
                   

In [114]:
# running the dataset against the above keywords to filter only bitcoin related tweets
elon_crypto = pd.DataFrame(columns = ['date', 'tweet'])

for i in range(len(elon_tweets)):
    cur = elon_tweets.loc[i].tweet
    cur = cur.lower().split()
    for j in cur:
            if j in crpyto_keywords:
                elon_crypto = elon_crypto.append({'date' : elon_tweets.loc[i].date, 'tweet' : elon_tweets.loc[i].tweet}, 
                ignore_index = True)

elon_crypto['date'] = pd.to_datetime(elon_crypto['date'], format='%Y-%m-%d')


            

In [115]:
# after cleaning the tweets and using regex to retrieve only elon musk's tweets, we filtered our list from 24172 to 135 tweets
elon_crypto.shape
# importing libraries for sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [116]:
scores_elon = []
# Declare variables for scores
compound_list1 = []
positive_list1 = []
negative_list1 = []
neutral_list1 = []
for i in range(elon_crypto['tweet'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(elon_crypto['tweet'][i])["compound"]
    pos = analyzer.polarity_scores(elon_crypto['tweet'][i])["pos"]
    neu = analyzer.polarity_scores(elon_crypto['tweet'][i])["neu"]
    neg = analyzer.polarity_scores(elon_crypto['tweet'][i])["neg"]
    
    scores_elon.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [117]:
sentiments_score_elon = pd.DataFrame.from_dict(scores_elon)
elon_crypto = elon_crypto.join(sentiments_score_elon)


In [118]:
elon_crypto = elon_crypto[elon_crypto['date']>='2019-01-01']

In [119]:
elon_crypto.reset_index(drop=True,inplace=True)


In [126]:
elon_crypto

elon_bit_doge = elon_crypto.merge(bitusd,on='date', how='left')



In [ ]:
elon_bit_doge.isnull().sum()

In [ ]:
# let's perform similar operations on bitcoin_2021 dataframe, from cleaning the data to performing sentiment analysis

bitcoin_tweets_2021.shape

In [ ]:
bitcoin_tweets_2021.columns

In [ ]:
bitcoin_tweets_2021 = bitcoin_tweets_2021[['user_name','date','text','hashtags','user_verified','user_followers'
                                          ,'user_friends','user_favourites']]

In [ ]:
bitcoin_tweets_2021 = bitcoin_tweets_2021[bitcoin_tweets_2021['user_verified'] ==True]
bitcoin_tweets_2021 = bitcoin_tweets_2021.reset_index(drop=True)

In [ ]:
print(bitcoin_tweets_2021['user_followers'].median())
print(bitcoin_tweets_2021['user_followers'].mean())
print(bitcoin_tweets_2021['user_followers'].mode())

In [ ]:
print(bitcoin_tweets_2021['user_favourites'].median())
print(bitcoin_tweets_2021['user_favourites'].mean())
print(bitcoin_tweets_2021['user_favourites'].mode())

In [ ]:
bitcoin_tweets_2021_new = bitcoin_tweets_2021.loc[(bitcoin_tweets_2021['user_followers']>=77581) &
                                                  (bitcoin_tweets_2021['user_favourites']>=6166)]
bitcoin_tweets_2021_new

In [ ]:
bitcoin_tweets_2021_new.reset_index(drop=True)

In [ ]:
#cleaning the tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z0-9]", " ")
    tweets = np.core.defchararray.replace(tweets, "#", " ")
    tweets = np.core.defchararray.replace(tweets, "$", " ")
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    
    
    
    return tweets

In [ ]:
bitcoin_tweets_2021_new['text'] = clean_tweets(bitcoin_tweets_2021_new['text'])

In [ ]:
bitcoin_tweets_2021_new.reset_index(inplace=True, drop=True)

In [ ]:
bitcoin_tweets_2021_new['text'] = bitcoin_tweets_2021_new['text'].str.lower()
bitcoin_tweets_2021_new['text'] = bitcoin_tweets_2021_new['text'].str.lstrip()

In [ ]:
bitcoin_tweets_2021_new.shape[0]

In [ ]:
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(bitcoin_tweets_2021_new['text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(bitcoin_tweets_2021_new['text'][i])["compound"]
    pos = analyzer.polarity_scores(bitcoin_tweets_2021_new['text'][i])["pos"]
    neu = analyzer.polarity_scores(bitcoin_tweets_2021_new['text'][i])["neu"]
    neg = analyzer.polarity_scores(bitcoin_tweets_2021_new['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
bitcoin_tweets_2021_new = bitcoin_tweets_2021_new.join(sentiments_score)
bitcoin_tweets_2021_new.head()



In [ ]:
bitcoin_tweets_2021_new.shape[0]

In [ ]:
bitcoin_tweets_2021_new.date = bitcoin_tweets_2021_new.date.str[:10] #tweets.date.str[:10]

In [ ]:
bitcoin_tweets_2021_new



In [123]:
bitusd = pd.read_csv('/Users/arjun/Downloads/BTCUSD.csv')

In [124]:
bitusd.rename(columns={'Date':'date'}, inplace=True)


In [125]:
bitusd['date'] = pd.to_datetime(bitusd['date'])

In [ ]:
# combining bitcoin historic price data to our tweets dataset
bitcoin_tweets_prices = bitcoin_tweets_2021_new.merge(bitusd, on='date', how='left')

In [ ]:
bitcoin_tweets_2021_new['date'] = pd.to_datetime(bitcoin_tweets_2021_new['date'])

In [ ]:
bitcoin_tweets_2021_new

NameError: name 'elon_bit_doge' is not defined

In [ ]:
bitcoin_tweets_prices.shape

In [ ]:
elon_bit_doge.isnull().sum()

In [ ]:
bitcoin_tweets_prices.isnull().sum()

In [ ]:
elon_bit_doge.columns

In [ ]:
bitcoin_tweets_prices.columns

In [127]:
elon_bit_doge_final = elon_bit_doge[['date', 'tweet', 'Compound', 'Open',
        'Close',  'Volume']]

In [ ]:
bitcoin_tweets_2021_prices_final = bitcoin_tweets_prices[[ 'date', 'text',
       'user_followers', 'user_favourites', 'Compound',
        'Open', 'Close',
        'Volume']]

In [ ]:
elon_bit_doge_final.shape

In [ ]:
bitcoin_tweets_2021_prices_final.shape

In [ ]:
elon_bit_doge_final.isnull().sum()

In [ ]:
bitcoin_tweets_2021_prices_final.isnull().sum()

In [ ]:
elon_bit_doge_final.columns

In [ ]:
bitcoin_tweets_2021_prices_final.columns

In [ ]:
elon_bit_doge_final.to_csv('elon_bit_doge_final.csv')

In [ ]:
bitcoin_tweets_2021_prices_final.to_csv('bitcoin_tweets_2021_prices_final.csv')

In [ ]:
import numpy as dragon
import pylab as p
import matplotlib.pyplot as plot
from collections import Counter
import re

#importing packages for the prediction of time-series data
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(x):


    #Determing rolling statistics
    rolmean = x.rolling(window=22,center=False).mean()

    rolstd = x.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plot.plot(x, color='blue',label='Original')
    mean = plot.plot(rolmean, color='red', label='Rolling Mean')
    std = plot.plot(rolstd, color='black', label = 'Rolling Std')
    plot.legend(loc='best')
    plot.title('Rolling Mean & Standard Deviation')
    plot.show(block=False)
    
    #Perform Dickey Fuller test    
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])
    pvalue=result[1]
    for key,value in result[4].items():
        if result[0]>value:
            print("The graph is non stationery")
            break
        else:
            print("The graph is stationery")
            break;
    print('Critical values:')
    for key,value in result[4].items():
        print('\t%s: %.3f ' % (key, value))
        
     


In [ ]:
test_stationarity(bitcoin_tweets_2021_prices_final['Close'])

In [ ]:
close = dragon.log(bitcoin_tweets_2021_prices_final['Close'])
plot.plot(close,color="green")
plot.show()

test_stationarity(close)

close

In [ ]:
close_diff = close - close.shift()
plot.plot(close_diff)
plot.show()

In [ ]:
close_diff.dropna(inplace=True)
test_stationarity(close_diff)

In [ ]:
close_diff

In [ ]:
bitcoin_tweets_2021_prices_final['Close']

In [132]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt

In [ ]:
features = ['user_followers', 'user_favourites', 'Compound', 'Open', 'Volume']

In [ ]:
y = bitcoin_tweets_2021_prices_final['Close']
x = bitcoin_tweets_2021_prices_final[features]

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state=1)

In [133]:
## since the dataset is very large spliting the dataset using the default 75:25 offered by random forrest model is ideal
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state=1,n_estimators=10)

In [ ]:
rf_model.fit(train_X, train_y)

In [ ]:
rf_pred = rf_model.predict(val_X)

In [ ]:
print(rf_model.predict(x))
bitcoin_tweets_2021_prices_final['Close']

In [134]:
from sklearn.metrics import mean_absolute_error
rf_val_mae = mean_absolute_error(val_y,rf_pred)
rf_val_mae

NameError: name 'val_y' is not defined

In [ ]:
# let's analyze the features user_followers and user_favourites to determine the correlation with price

In [ ]:
plot.scatter(bitcoin_tweets_2021_prices_final['Close'],bitcoin_tweets_2021_prices_final['user_followers'])
plt.show()

In [ ]:
plot.scatter(bitcoin_tweets_2021_prices_final['Close'],bitcoin_tweets_2021_prices_final['user_favourites'])
plt.show()

On the basis of the plots above, there does not seem to be much correlation between user_favourutes,user_followers 
and Closing price of bitcoin, lets exclude these features and test our model again

In [135]:
features_updated = ['Compound', 'Open', 'Volume']

x_upd = bitcoin_tweets_2021_prices_final[features_updated]

NameError: name 'bitcoin_tweets_2021_prices_final' is not defined

In [ ]:
train_X_upd, val_X_upd, train_y_upd, val_y_upd = train_test_split(x_upd, y, random_state=1)

In [ ]:
rf_model_upd = RandomForestRegressor(random_state=1,n_estimators=10)

In [ ]:
rf_model_upd.fit(train_X_upd, train_y_upd)

In [ ]:
rf_pred_upd = rf_model_upd.predict(val_X_upd)

In [ ]:
print(rf_model_upd.predict(x_upd))
bitcoin_tweets_2021_prices_final['Close']

In [ ]:
rf_val_mae_upd = mean_absolute_error(val_y_upd,rf_pred_upd)
rf_val_mae_upd

In [ ]:
# lets perform the prediction for elon musks tweets

In [128]:
elon_bit_doge_final.shape[0]

97

In [200]:
elon_bit_doge_final.columns

Index(['date', 'tweet', 'Compound', 'Open', 'Close', 'Volume'], dtype='object')

In [136]:
features_elon = ['Compound','Open', 'Volume']

In [137]:
y_elon = elon_bit_doge_final['Close']
x_elon = elon_bit_doge_final[features_elon]

In [138]:
train_X_elon, val_X_elon, train_y_elon, val_y_elon = train_test_split(x_elon, y_elon, random_state=1)

In [195]:
rf_model_elon = RandomForestRegressor(random_state=1, n_estimators=90, max_depth=6)

In [196]:
rf_model_elon.fit(train_X_elon, train_y_elon)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=90,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [197]:
rf_pred_elon = rf_model_elon.predict(val_X_elon)

In [198]:
print(rf_model_elon.predict(x_elon))
elon_bit_doge_final['Close']

[ 8094.07820904 10965.95388808  4909.7674645   4939.656999
  4918.28546038 22610.27503479 16580.65315739 11975.57488081
 10987.45782712 11216.72347033 10365.08489624  9042.27615272
  8310.58296699  9303.20093728  8094.07820904 10965.95388808
  4909.7674645   4939.656999    4918.28546038 48058.32510947
 49417.38416389 45780.10644953 46783.5675504  57367.12515622
 57402.76107202 57409.1408608  62772.1519749  60686.46289693
 60415.31403856 61508.78253046 54820.66940717 49355.72740301
 48548.66611222 43502.17893451 46279.36264076 32039.30055773
 32264.30376945 32525.51603617 32525.51603617 32473.33215656
 32296.36978387 32116.79838631 32116.79838631 39682.38923883
 40788.56244202 39817.54633244 40203.92296686 40203.92296686
 40352.40869819 46616.27664925 46633.03162611 46572.0118923
 48162.14057709 56894.01851527 56452.45208413 58372.82798543
 56940.21180186 57334.90752532 55327.76614681 48848.19081904
 62980.27994812 62980.27994812 58372.6521246  57629.7443875
 56894.01851527 59370.562731

0      8336.555664
1     11011.102539
2      4879.877930
3      4879.877930
4      4879.877930
          ...     
92    43099.699219
93    39400.585938
94    42451.789063
95    36954.003906
96    43099.699219
Name: Close, Length: 97, dtype: float64

In [199]:
rf_val_mae_elon = mean_absolute_error(val_y_elon,rf_pred_elon)
rf_val_mae_elon

729.1304314032616

the mae for elon musks tweets is not the greatest, partly because we considered day by day close data for bitcoin most of the tweets by musk effect intraday(within a day) trading. 

In [ ]:
## lets use linearregression to perform the same prediction
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(train_X, train_y)

In [ ]:
model.score(val_X, val_y)

In [ ]:
future_set = val_y

In [ ]:
prediction = model.predict(val_X)

In [ ]:
mae_bitcoin_lr = mean_absolute_error(val_y,prediction)

In [ ]:
mae_bitcoin_lr

Let's perform kfold validation on elon musk dataset as the dataset is limited

In [79]:
def rfr_model(X, y):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
# Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores

In [80]:
elon = pd.read_csv("elon_bit_doge_final.csv")

In [81]:
elon_fet = ['Compound','Open','Volume']
elon_price = elon['Close']

fet = elon[elon_fet]

In [84]:
rfr =rfr_model(fet,elon_price)

print(rfr)

/Users/arjun/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[-2048.38169093  -896.26083709 -1356.94222757 -1238.80833122
 -3418.88508226  -752.60616106 -1599.17119843 -1745.45125246
 -4310.16788636 -3880.98399135]


In [83]:
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [85]:
scores = cross_val_score(rfr, fet, elon_price, cv=10, scoring='neg_mean_absolute_error')

TypeError: estimator should be an estimator implementing 'fit' method, array([-2048.38169093,  -896.26083709, -1356.94222757, -1238.80833122,
       -3418.88508226,  -752.60616106, -1599.17119843, -1745.45125246,
       -4310.16788636, -3880.98399135]) was passed

In [55]:
predictions = cross_val_predict(rfr, elon_fet, cv=3)

TypeError: Cannot clone object 'array([-2015.44120798,  -816.45893733, -1372.48862109, -1035.05907168,
       -3370.94465553,  -664.42894126, -1334.6409081 , -1754.83843109,
       -4067.3808656 , -3767.39502778])' (type <class 'numpy.ndarray'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

In [61]:
rf_model_e = RandomForestRegressor(random_state=1,n_estimators=10)

In [70]:
accuracy = cross_val_score(rf_model_e, fet, elon_price, scoring='neg_mean_absolute_error', cv = 10)
print(accuracy)

[-2316.05792001  -925.47012705 -1512.69242204 -1236.608047
 -3172.26750008  -778.56304703 -1273.71933594 -1907.14279507
 -4038.54919732 -4308.20937481]


In [71]:
predictions = cross_val_predict(rf_model_e, fet, elon_price, cv=10)

In [78]:
print(predictions)
print(elon['Close'])

[ 8448.3440917  9788.5992187  5225.5457034  5225.5457034  5225.5457034
 32923.7582034  9957.7828125 10058.5637695  9795.1549804  9714.4471191
 11143.8722659  9337.9585938  8987.3106448  9578.4138674  9416.1716797
 10746.7934571  5493.0003909  5493.0003909  5838.6681643 46947.3480471
 48776.7132815 45338.4164063 47044.0511721 55641.4648437 55941.3566406
 56115.5234376 62003.5031251 59302.316406  59195.316406  62110.5031251
 56726.2890626 50402.0722661 49093.2093754 43419.7742191 46218.9882816
 33096.089844  33096.089844  32218.0806643 32218.0806643 33096.089844
 34049.9261721 34049.9261721 34049.9261721 45026.0785159 45026.0785159
 42725.0929689 37867.0749998 37867.0749998 37867.0749998 47451.0355472
 46753.9281253 46615.7980472 48346.5882813 56816.6601564 58232.3816409
 58457.1285159 56982.5929688 57963.005469  54831.6117189 50376.2003908
 59655.7800781 59655.7800781 57277.4410155 56610.5644531 56672.8972658
 58955.4625    58955.4625    58955.4625    58955.4625    58955.4625
 50493.897

In [91]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
#read csv file



accuracy = cross_val_score(rf_model_e, fet, elon_price, scoring='neg_mean_absolute_error', cv = 10)
print(accuracy)
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is:",accuracy.mean())

[-2316.05792001  -925.47012705 -1512.69242204 -1236.608047
 -3172.26750008  -778.56304703 -1273.71933594 -1907.14279507
 -4038.54919732 -4308.20937481]
Accuracy of Model with Cross Validation is: -2146.927976635002


In [92]:
gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
grid_result = gsc.fit(fet, elon_price)
best_params = grid_result.best_params_
    

/Users/arjun/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [94]:
best_params

{'max_depth': 6, 'n_estimators': 50}

In [95]:
rf = RandomForestRegressor(random_state=1,n_estimators=50, max_depth=6)

In [96]:
accuracy_rf = cross_val_score(rf, fet, elon_price, scoring='neg_mean_absolute_error', cv = 11)

In [97]:
accuracy_rf

array([-1997.92509075,  -700.58580507, -1292.8758022 , -1492.93642142,
       -1466.16461151, -1828.63585314, -1236.38206983, -1419.29608445,
       -1721.01425217, -2362.66140207, -3395.3238684 ])

In [98]:
accuracy_rf_1 = cross_val_predict(rf,fet,elon_price,cv=11)

In [101]:
len(accuracy_rf_1)

97

In [100]:
print(mean_absolute_error(accuracy_rf_1,elon_price))

1695.528103902383
